# LLM Observability with LangWatch

Jaeger is a general-purpose observability platform, but there are specialized tools designed specifically for LLMs.

Open-source options:

- Langfuse: https://langfuse.com/
- LangWatch: https://langwatch.ai/
- Arize Phoenix: https://phoenix.arize.com/

Closed-source options:

- Arize: https://arize.com/

## LangWatch

In this lesson I'll use LangWatch. It's just personal preference, and we already saw LangWatch Scenario in the testing section.

They have documentation about PydanticAI integration: https://docs.langwatch.ai/integration/python/integrations/pydantic-ai

Here's their demo showcasing the platform capabilities: https://www.youtube.com/watch?v=lsS06UXZB2U

## Setting Up LangWatch Locally

LangWatch provides clear instructions for local deployment: https://docs.langwatch.ai/self-hosting/docker-compose

Let's set it up:

```bash
git clone https://github.com/langwatch/langwatch.git
cd langwatch
cp langwatch/.env.example langwatch/.env
docker-compose up --build
```

Add langwatch/ to your .gitignore to avoid committing the cloned repository.

```bash
echo langwatch >> .gitignore
```

You can also use their cloud version if you prefer not to run it locally.

Open the interface: http://localhost:5560

Next, add the LangWatch configuration to your `.env` file:

```text
LANGWATCH_ENDPOINT="http://localhost:5560"
LANGWATCH_API_KEY="some-key"
```

Install the LangWatch SDK:


In [ ]:
!uv add langwatch

## Instrumenting Your Application

I previously mentioned that it's possible to instrument PydanticAI without Logfire. This is exactly what LangWatch does:

In [ ]:
import langwatch
from dotenv import load_dotenv
from pydantic_ai import Agent

load_dotenv()
langwatch.setup()
Agent.instrument_all()


Add the @langwatch.trace() decorator to your main function:

In [ ]:
# main.py
@langwatch.trace()
async def main():
    # ...existing code...

Run your application and explore the results in the LangWatch UI!

## Running Docker in /tmp (for Codespaces)

If you also (like me) get "no free space left on device", this is what you can do to use the /tmp folder.

Create a directory for Docker data:

```bash
sudo mkdir -p /tmp/docker-data
sudo chmod 711 /tmp/docker-data
```

Stop the docker process:

```bash
sudo pkill dockerd
sudo rm -f /var/run/docker.pid
```

Start again with the new data root:

```bash
export DOCKER_TMPDIR=/tmp/docker-data
sudo dockerd --host=unix:///var/run/docker.sock
```

In another terminal, verify that it worked:

```bash
docker info | grep "Docker Root Dir"
```

You should see the new directory. Now run docker compose.

If you run fails with out of memory, consider giving opensearch more RAM:

```yaml
opensearch:
  image: langwatch/opensearch-lite:latest
  environment:
    # update from 256m to 512m
    - "OPENSEARCH_JAVA_OPTS=-Xms512m -Xmx512m ..." 
    # ...
  # replace
  # deploy:
  #   resources:
  #     limits:
  #       memory: 256m
  #       cpus: "1.0"
  # to this:
  mem_limit: 1g
  ```